In [1]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
import albumentations as A

In [2]:
# Set random seeds
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Enable mixed precision
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [3]:
# Data paths
BASE_DIR = "D:\\Ashutosh\\Herbs\\Cleanede_Data"
TRAIN_DIR = os.path.join(BASE_DIR, "Train")
VAL_DIR = os.path.join(BASE_DIR, "Val")
OUTPUT_DIR = "D:\\Ashutosh\\Herbs\\Trained_Output_EfficientNetV2"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
# Image parameters
IMAGE_SIZE = (224, 224)  # EfficientNetV2S default size
BATCH_SIZE = 32
NUM_CLASSES = 109

In [5]:
# Augmentation
train_transform = A.Compose([
    A.Resize(*IMAGE_SIZE),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.ToFloat()
])
val_transform = A.Compose([
    A.Resize(*IMAGE_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    A.ToFloat()
])

c:\Users\KU_AI_M\miniconda3\envs\tf\lib\site-packages\albumentations\core\validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [7]:
# Data pipeline
def load_and_preprocess_image(file_path, label, is_training=True):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    def apply_augmentation(image):
        return train_transform(image=image.numpy())['image'] if is_training else val_transform(image=image.numpy())['image']
    img = tf.py_function(apply_augmentation, [img], tf.float32)
    img.set_shape([*IMAGE_SIZE, 3])
    return img, label

def create_dataset(directory, is_training=True):
    file_paths, labels = [], []
    class_names = sorted(os.listdir(directory))
    if len(class_names) != NUM_CLASSES:
        raise ValueError(f"Expected {NUM_CLASSES} classes, found {len(class_names)}")
    class_to_idx = {name: idx for idx, name in enumerate(class_names)}
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name)
        if not os.path.isdir(class_dir):
            continue
        for fname in os.listdir(class_dir):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_paths.append(os.path.join(class_dir, fname))
                labels.append(class_to_idx[class_name])
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, tf.keras.utils.to_categorical(labels, NUM_CLASSES)))
    dataset = dataset.map(lambda x, y: load_and_preprocess_image(x, y, is_training), num_parallel_calls=tf.data.AUTOTUNE)
    if is_training:
        dataset = dataset.shuffle(buffer_size=1000, seed=SEED)
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset, len(file_paths), np.bincount(labels)

# Debug raw images and class distribution
train_file_paths = []
class_names = sorted(os.listdir(TRAIN_DIR))
for class_name in class_names:
    class_dir = os.path.join(TRAIN_DIR, class_name)
    if not os.path.isdir(class_dir):
        continue
    for fname in os.listdir(class_dir):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            train_file_paths.append(os.path.join(class_dir, fname))
for img_path in tf.data.Dataset.from_tensor_slices(train_file_paths).take(5).as_numpy_iterator():
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    print(f"Raw image {img_path.decode()}: min={tf.reduce_min(img)}, max={tf.reduce_max(img)}")

train_dataset, train_size, train_class_counts = create_dataset(TRAIN_DIR, is_training=True)
val_dataset, val_size, val_class_counts = create_dataset(VAL_DIR, is_training=False)
print(f"Training samples: {train_size}, Validation samples: {val_size}")
print("Training class distribution:", train_class_counts)
print("Validation class distribution:", val_class_counts)

Raw image D:\Ashutosh\Herbs\Cleanede_Data\Train\Achillea Millefolium\0088bfd6-6481-4b32-b62e-b92ae642ccd0.jpg: min=0.0, max=1.0
Raw image D:\Ashutosh\Herbs\Cleanede_Data\Train\Achillea Millefolium\016258ca-9e71-479a-8e3e-dd365ce13e40.jpg: min=0.0, max=0.8862745761871338
Raw image D:\Ashutosh\Herbs\Cleanede_Data\Train\Achillea Millefolium\0172e1ad-437b-4814-94a7-6946c01b1a25.jpg: min=0.0, max=1.0
Raw image D:\Ashutosh\Herbs\Cleanede_Data\Train\Achillea Millefolium\018c0c14-4831-4da2-a77e-c2409b98bf36.jpg: min=0.0, max=1.0
Raw image D:\Ashutosh\Herbs\Cleanede_Data\Train\Achillea Millefolium\01aeff5a-6cf3-4b5c-834b-34b4e2a43d15.jpg: min=0.0, max=1.0
Training samples: 60236, Validation samples: 15059
Training class distribution: [544 563 567 563 553 553 552 553 541 549 532 561 582 557 561 556 552 568
 559 548 536 580 573 570 559 555 566 558 554 542 560 511 559 549 559 557
 563 534 572 563 550 555 545 506 555 571 555 562 581 559 517 553 472 542
 582 566 571 534 555 572 434 568 546 577 529 5

In [8]:
# Compute class weights
class_weights = {i: train_size / (NUM_CLASSES * count) for i, count in enumerate(train_class_counts)}
print("Class weights:", class_weights)

Class weights: {0: 1.0158526713437668, 1: 0.981569899131455, 2: 0.9746452437583936, 3: 0.981569899131455, 4: 0.9993198068915174, 5: 0.9993198068915174, 6: 1.0011301688605239, 7: 0.9993198068915174, 8: 1.0214858654547305, 9: 1.006600825520964, 10: 1.0387666413740775, 11: 0.9850692570606224, 12: 0.9495255209811154, 13: 0.9921433630359231, 14: 0.9850692570606224, 15: 0.9939277935449805, 16: 1.0011301688605239, 17: 0.9729293190334669, 18: 0.9885936551180844, 19: 1.0084376883412576, 20: 1.031014651513077, 21: 0.952799746915533, 22: 0.964439534399667, 23: 0.9695155319491389, 24: 0.9885936551180844, 25: 0.9957186544342508, 26: 0.9763672318215709, 27: 0.9903653283351419, 28: 0.9975159805252873, 29: 1.0196012051863639, 30: 0.9868283093053736, 31: 1.0814556814305463, 32: 0.9885936551180844, 33: 1.006600825520964, 34: 0.9885936551180844, 35: 0.9921433630359231, 36: 0.981569899131455, 37: 1.0348761296086315, 38: 0.9661256175017643, 39: 0.981569899131455, 40: 1.0047706422018348, 41: 0.9957186544342

In [9]:
# Model definition
feature_extractor = hub.KerasLayer(
    "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2",
    trainable=False
)
inputs = tf.keras.Input(shape=(*IMAGE_SIZE, 3))
x = layers.Lambda(lambda x: feature_extractor(x))(inputs)
x = layers.Dense(512, activation='relu', dtype='float32')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)
model = Model(inputs=inputs, outputs=outputs)

In [10]:
# Debug: Inspect batch and features
for images, labels in train_dataset.take(1):
    print("Batch shape:", images.shape, labels.shape)
    print("Sample labels:", labels[:5])
    print("Image min/max:", tf.reduce_min(images), tf.reduce_max(images))
    feature_model = Model(inputs=model.input, outputs=model.layers[1].output)
    features = feature_model.predict(images)
    print("Feature shape:", features.shape, "Feature norm:", np.linalg.norm(features))

Batch shape: (32, 224, 224, 3) (32, 109)
Sample labels: tf.Tensor(
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [11]:
# Compile
model.compile(
    optimizer=Adam(learning_rate=0.002, clipnorm=1.0),
    loss=CategoricalCrossentropy(),
    metrics=['accuracy', 'top_k_categorical_accuracy']
)

In [12]:
callbacks = [
    ModelCheckpoint(os.path.join(OUTPUT_DIR, "HerbClassification_BEST_Stage1.h5"), monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    CSVLogger(os.path.join(OUTPUT_DIR, f"training_log_stage1_{int(time.time())}.csv"))
]

In [13]:
history_stage1 = model.fit(
    train_dataset,
    epochs=20,
    steps_per_epoch=(train_size + BATCH_SIZE - 1) // BATCH_SIZE,
    validation_data=val_dataset,
    validation_steps=(val_size + BATCH_SIZE - 1) // BATCH_SIZE,
    callbacks=callbacks,
    class_weight=class_weights  # Add class weights
)

Epoch 1/20


1883/1883 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.0888 - loss: 4.3080 - top_k_categorical_accuracy: 0.2704
Epoch 1: val_accuracy improved from -inf to 0.00923, saving model to D:\Ashutosh\Herbs\Trained_Output_EfficientNetV2\HerbClassification_BEST_Stage1.h5


1883/1883 ━━━━━━━━━━━━━━━━━━━━ 1173s 618ms/step - accuracy: 0.0888 - loss: 4.3083 - top_k_categorical_accuracy: 0.2703 - val_accuracy: 0.0092 - val_loss: 4.7554 - val_top_k_categorical_accuracy: 0.0494 - learning_rate: 0.0020
Epoch 2/20
1883/1883 ━━━━━━━━━━━━━━━━━━━━ 0s 501ms/step - accuracy: 0.0231 - loss: 4.4549 - top_k_categorical_accuracy: 0.1047
Epoch 2: val_accuracy did not improve from 0.00923
1883/1883 ━━━━━━━━━━━━━━━━━━━━ 1172s 622ms/step - accuracy: 0.0231 - loss: 4.4551 - top_k_categorical_accuracy: 0.1046 - val_accuracy: 0.0091 - val_loss: 4.6980 - val_top_k_categorical_accuracy: 0.0450 - learning_rate: 0.0020
Epoch 3/20
1883/1883 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.0230 - loss: 4.5853 - top_k_categorical_accuracy: 0.1114
Epoch 3: val_accuracy improved from 0.00923 to 0.00963, saving model to D:\Ashutosh\Herbs\Trained_Output_EfficientNetV2\HerbClassification_BEST_Stage1.h5


1883/1883 ━━━━━━━━━━━━━━━━━━━━ 1164s 617ms/step - accuracy: 0.0230 - loss: 4.5854 - top_k_categorical_accuracy: 0.1113 - val_accuracy: 0.0096 - val_loss: 4.6944 - val_top_k_categorical_accuracy: 0.0467 - learning_rate: 0.0020
Epoch 4/20
1883/1883 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step - accuracy: 0.0153 - loss: 4.6693 - top_k_categorical_accuracy: 0.0807
Epoch 4: val_accuracy did not improve from 0.00963
1883/1883 ━━━━━━━━━━━━━━━━━━━━ 1155s 612ms/step - accuracy: 0.0153 - loss: 4.6693 - top_k_categorical_accuracy: 0.0807 - val_accuracy: 0.0096 - val_loss: 4.6998 - val_top_k_categorical_accuracy: 0.0432 - learning_rate: 0.0020
Epoch 5/20
 681/1883 ━━━━━━━━━━━━━━━━━━━━ 10:05 503ms/step - accuracy: 0.0161 - loss: 4.6975 - top_k_categorical_accuracy: 0.0796

KeyboardInterrupt: 

In [ ]:
# Debug: Predict on a batch
for images, labels in val_dataset.take(1):
    preds = model.predict(images)
    print("Sample predictions:", preds[:5])
    print("Sample true labels:", labels[:5])

In [ ]:
# Plot history
def plot_history(history, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.plot(history.history['accuracy'], label='Train Acc')
    ax1.plot(history.history['val_accuracy'], label='Val Acc')
    ax1.set_title(f'{title} - Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    ax2.set_title(f'{title} - Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    plt.tight_layout()
    plt.show()

plot_history(history_stage1, "Stage 1: Head Training")

In [ ]:
# Stage 2: Unfreeze and fine-tune all layers
feature_extractor.trainable = True
model.compile(
    optimizer=AdamW(learning_rate=0.0005, weight_decay=0.0001),  # Lower LR for fine-tuning
    loss=CategoricalCrossentropy(label_smoothing=0.1),  # Reintroduce smoothing
    metrics=['accuracy']
)

In [ ]:
callbacks = [
    ModelCheckpoint(os.path.join(OUTPUT_DIR, "HerbClassification_BEST_Stage2.h5"), monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1),
    CSVLogger(os.path.join(OUTPUT_DIR, f"training_log_stage2_{int(time.time())}.csv"))
]

In [ ]:
history_stage2 = model.fit(
    train_dataset,
    epochs=50,
    steps_per_epoch=(train_size + BATCH_SIZE - 1) // BATCH_SIZE,
    validation_data=val_dataset,
    validation_steps=(val_size + BATCH_SIZE - 1) // BATCH_SIZE,
    callbacks=callbacks
)

In [ ]:
# Save final model
model.save(os.path.join(OUTPUT_DIR, "HerbClassification_Final.h5"))

In [ ]:
# Evaluate
val_loss, val_accuracy = model.evaluate(val_dataset, steps=(val_size + BATCH_SIZE - 1) // BATCH_SIZE)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
# Plot history
def plot_history(history, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Val Accuracy')
    ax1.set_title(f'{title} - Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    ax2.set_title(f'{title} - Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    plt.tight_layout()
    plt.show()

plot_history(history_stage1, "Stage 1: Head Training")
plot_history(history_stage2, "Stage 2: Full Fine-Tuning")